# Generating Genome Feature Tracks

In this notebook, I'll use the [NCBI assembly](https://www.ncbi.nlm.nih.gov/assembly/GCF_902806645.1/) and [NCBI Annotation Release 102](https://www.ncbi.nlm.nih.gov/genome/annotation_euk/Crassostrea_gigas/102/) to genome feature tracks for the Roslin *C. gigas* genome.

## 0. Set working directory and variables

In [1]:
!pwd

/Users/yaaminivenkataraman/Documents/ceabigr/code


In [3]:
#!mkdir ../genome-features/

In [2]:
cd ../genome-features/

/Users/yaaminivenkataraman/Documents/ceabigr/genome-features


In [5]:
!which bedtools

/opt/homebrew/bin/bedtools


In [30]:
bedtoolsDirectory = "/opt/homebrew/bin"

## 1. Download NCBI assembly

I downloaded the GFF from [this link](https://www.ncbi.nlm.nih.gov/genome/?term=txid6565[orgn]).

In [8]:
!head -n100 GCF_002022765.2_C_virginica-3.0_genomic.gff

##gff-version 3
#!gff-spec-version 1.21
#!processor NCBI annotwriter
#!genome-build C_virginica-3.0
#!genome-build-accession NCBI_Assembly:GCF_002022765.2
#!annotation-source NCBI Crassostrea virginica Annotation Release 100
##sequence-region NC_035780.1 1 65668440
##species https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=6565
NC_035780.1	RefSeq	region	1	65668440	.	+	.	ID=NC_035780.1:1..65668440;Dbxref=taxon:6565;Name=1;chromosome=1;collection-date=22-Mar-2015;country=USA;gbkey=Src;genome=chromosome;isolate=RU13XGHG1-28;isolation-source=Rutgers Haskin Shellfish Research Laboratory inbred lines (NJ);mol_type=genomic DNA;tissue-type=whole sample
NC_035780.1	Gnomon	gene	13578	14594	.	+	.	ID=gene-LOC111116054;Dbxref=GeneID:111116054;Name=LOC111116054;gbkey=Gene;gene=LOC111116054;gene_biotype=lncRNA
NC_035780.1	Gnomon	lnc_RNA	13578	14594	.	+	.	ID=rna-XR_002636969.1;Parent=gene-LOC111116054;Dbxref=GeneID:111116054,Genbank:XR_002636969.1;Name=XR_002636969.1;gbkey=ncRNA;gene=LOC111

## 2. Prepare for feature track creation

Before I pull out feature tracks, I need to know which databases were used for annotation, which features I can expect and how many of them there are, and identify chromosome lengths.

### 2a. Annotation information

In [9]:
#Database identifiers for extracting features
!cut -f2 GCF_002022765.2_C_virginica-3.0_genomic.gff | sort | uniq -c | tail

   1 ##sequence-region NC_035784.1 1 98698416
   1 ##sequence-region NC_035785.1 1 51258098
   1 ##sequence-region NC_035786.1 1 57830854
   1 ##sequence-region NC_035787.1 1 75944018
   1 ##sequence-region NC_035788.1 1 104168038
   1 ##sequence-region NC_035789.1 1 32650045
  11 ##species https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=6565
1482188 Gnomon
29345 RefSeq
1739 tRNAscan-SE


In [10]:
#Count the number of unique features in the GFF
!cut -f3 GCF_002022765.2_C_virginica-3.0_genomic.gff | sort | uniq -c

   1 #!annotation-source NCBI Crassostrea virginica Annotation Release 100
   1 #!genome-build C_virginica-3.0
   1 #!genome-build-accession NCBI_Assembly:GCF_002022765.2
   1 #!gff-spec-version 1.21
   1 #!processor NCBI annotwriter
   1 ###
   1 ##gff-version 3
   1 ##sequence-region NC_007175.2 1 17244
   1 ##sequence-region NC_035780.1 1 65668440
   1 ##sequence-region NC_035781.1 1 61752955
   1 ##sequence-region NC_035782.1 1 77061148
   1 ##sequence-region NC_035783.1 1 59691872
   1 ##sequence-region NC_035784.1 1 98698416
   1 ##sequence-region NC_035785.1 1 51258098
   1 ##sequence-region NC_035786.1 1 57830854
   1 ##sequence-region NC_035787.1 1 75944018
   1 ##sequence-region NC_035788.1 1 104168038
   1 ##sequence-region NC_035789.1 1 32650045
  11 ##species https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=6565
645368 CDS
29258 cDNA_match
731916 exon
38838 gene
4750 lnc_RNA
60201 mRNA
 667 pseudogene
   2 rRNA
  11 region
 587 tRNA
1674 transcript


### 2b. Chromosome lengths

In [11]:
#Obtained the full FASTA from this link: https://www.ncbi.nlm.nih.gov/genome/?term=txid6565[orgn]
#Check information
!head GCF_002022765.2_C_virginica-3.0_genomic.fna

>NC_035780.1 Crassostrea virginica isolate RU13XGHG1-28 chromosome 1, C_virginica-3.0, whole genome shotgun sequence
tgacacatatataaagttgaagTCCATACGTAAGAAACTCTGTGAGATATTAACCGAAAACCTTTTGAATCTTTacgaaa
aatatacatgttgcGCCAACTGGCGTAAATCAAAACCGGAAGCAGTAAGCATGTCGTGTTTAGTGTCTATCAAATGGACC
GGGGGAGTTCTAGTACATATCCAAAGATAAGGGCAATACATAAAATACTCGCAAAGTTATTGACCGtcaaagttgatgta
cttttagaaaaaaataatggaaaatgtggcTTTAGTGGAACGGCatcaatgtaaatttaaaatagcaggGTTTGCGTTTG
AATTAAAACATCGTGTTTGGTGTCGTTAGAAAGGTCTATTCCAGTTCTATAACATATCTAAAGGTCAGGTCAATCCgtta
atttataaaagagaTATGGGCGATCGCGGCATGAGTACTACATGACACAGAGTTACTTGCTCTTTGCTACTTCAGCGTTT
CCGGAAGCGTAGTTTTTTTCGTGCGTTTATTCCTTGCAGATAGCCAAGCAATTCCACAAGAATTGAACTCATTTGGCATT
AaacttcttgaaaaaataacaaattctttcttttctcatatCAGGTGGATgtcgagtactttgattctaccgggcataga
gtagcaacagtactttcagtaccgtgattttgctaccaatcataggactgaaagtactgtgtatagaccaatcacagtac


In [16]:
#Extract chr and sequence length information
!awk '$0 ~ ">" {print c; c=0;printf substr($0,2,14) "\t"; } $0 !~ ">" {c+=length($0);} END { print c; }' \
GCF_002022765.2_C_virginica-3.0_genomic.fna \
| sed 's/Cr//g' \
| awk '{print $1"\t"$2}' \
| tail -n +2 \
> C_virginica-3.0-sequence-lengths.txt

In [20]:
#11 chr total including mitochondrial information
!head -n11 C_virginica-3.0-sequence-lengths.txt

NC_035780.1	65668440
NC_035781.1	61752955
NC_035782.1	77061148
NC_035783.1	59691872
NC_035784.1	98698416
NC_035785.1	51258098
NC_035786.1	57830854
NC_035787.1	75944018
NC_035788.1	104168038
NC_035789.1	32650045
NC_007175.2	17244


In [33]:
#New file with chr names only
!cut -f1 C_virginica-3.0-sequence-lengths.txt \
> C_virginica-3.0-chr.txt

## 2. Generate genome feature tracks

I will extract CDS, exon, gene, lncRNA and mRNA tracks. I can then use those existing tracks to produce intron and  intergenic tracks, as well as 1 kb upstream and downstream flanking regions with `bedtools`. I will also use the RepeatMasker output from NCBI for my transposable element track.

In [31]:
!{bedtoolsDirectory}/bedtools --version

bedtools v2.30.0


### 2a. Gene

In [35]:
#Isolate gene entries from multiple annotation databses. Tab mus be included between database and feature
#Sort output for downstream use
#Include chromosome name information
!grep -e "Gnomon	gene" -e "RefSeq	gene" -e "tRNAscan-SE	gene" \
GCF_002022765.2_C_virginica-3.0_genomic.gff \
| {bedtoolsDirectory}/sortBed \
-faidx C_virginica-3.0-chr.txt \
> C_virginica-3.0-gene.gff

In [36]:
!head C_virginica-3.0-gene.gff
!wc -l C_virginica-3.0-gene.gff

NC_035780.1	Gnomon	gene	13578	14594	.	+	.	ID=gene-LOC111116054;Dbxref=GeneID:111116054;Name=LOC111116054;gbkey=Gene;gene=LOC111116054;gene_biotype=lncRNA
NC_035780.1	Gnomon	gene	28961	33324	.	+	.	ID=gene-LOC111126949;Dbxref=GeneID:111126949;Name=LOC111126949;gbkey=Gene;gene=LOC111126949;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	43111	66897	.	-	.	ID=gene-LOC111110729;Dbxref=GeneID:111110729;Name=LOC111110729;gbkey=Gene;gene=LOC111110729;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	85606	95254	.	-	.	ID=gene-LOC111112434;Dbxref=GeneID:111112434;Name=LOC111112434;gbkey=Gene;gene=LOC111112434;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	99840	106460	.	+	.	ID=gene-LOC111120752;Dbxref=GeneID:111120752;Name=LOC111120752;gbkey=Gene;gene=LOC111120752;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	108305	110077	.	-	.	ID=gene-LOC111128944;Dbxref=GeneID:111128944;Name=LOC111128944;gbkey=Gene;gene=LOC111128944;gene_biotype=protein_coding;partial=true;start_range=.,108305

### 2b. CDS

In [38]:
!grep -e "Gnomon	CDS" -e "RefSeq	CDS" -e "tRNAscan-SE	CDS" \
GCF_002022765.2_C_virginica-3.0_genomic.gff \
| {bedtoolsDirectory}/sortBed \
-faidx C_virginica-3.0-chr.txt \
> C_virginica-3.0-CDS.gff

In [39]:
!head C_virginica-3.0-CDS.gff
!wc -l C_virginica-3.0-CDS.gff

NC_035780.1	Gnomon	CDS	30535	31557	.	+	0	ID=cds-XP_022327646.1;Parent=rna-XM_022471938.1;Dbxref=GeneID:111126949,Genbank:XP_022327646.1;Name=XP_022327646.1;gbkey=CDS;gene=LOC111126949;product=UNC5C-like protein;protein_id=XP_022327646.1
NC_035780.1	Gnomon	CDS	31736	31887	.	+	0	ID=cds-XP_022327646.1;Parent=rna-XM_022471938.1;Dbxref=GeneID:111126949,Genbank:XP_022327646.1;Name=XP_022327646.1;gbkey=CDS;gene=LOC111126949;product=UNC5C-like protein;protein_id=XP_022327646.1
NC_035780.1	Gnomon	CDS	31977	32565	.	+	1	ID=cds-XP_022327646.1;Parent=rna-XM_022471938.1;Dbxref=GeneID:111126949,Genbank:XP_022327646.1;Name=XP_022327646.1;gbkey=CDS;gene=LOC111126949;product=UNC5C-like protein;protein_id=XP_022327646.1
NC_035780.1	Gnomon	CDS	32959	33204	.	+	0	ID=cds-XP_022327646.1;Parent=rna-XM_022471938.1;Dbxref=GeneID:111126949,Genbank:XP_022327646.1;Name=XP_022327646.1;gbkey=CDS;gene=LOC111126949;product=UNC5C-like protein;protein_id=XP_022327646.1
NC_035780.1	Gnomon	CDS	43262	44358	.	-	2	ID=cds-XP_0

### 2c. Exon

In [40]:
!grep -e "Gnomon	exon" -e "RefSeq	exon" -e "tRNAscan-SE	exon" \
GCF_002022765.2_C_virginica-3.0_genomic.gff \
| {bedtoolsDirectory}/sortBed \
-faidx C_virginica-3.0-chr.txt \
> C_virginica-3.0-exon.gff

In [41]:
!head C_virginica-3.0-exon.gff
!wc -l C_virginica-3.0-exon.gff

NC_035780.1	Gnomon	exon	13578	13603	.	+	.	ID=exon-XR_002636969.1-1;Parent=rna-XR_002636969.1;Dbxref=GeneID:111116054,Genbank:XR_002636969.1;gbkey=ncRNA;gene=LOC111116054;product=uncharacterized LOC111116054;transcript_id=XR_002636969.1
NC_035780.1	Gnomon	exon	14237	14290	.	+	.	ID=exon-XR_002636969.1-2;Parent=rna-XR_002636969.1;Dbxref=GeneID:111116054,Genbank:XR_002636969.1;gbkey=ncRNA;gene=LOC111116054;product=uncharacterized LOC111116054;transcript_id=XR_002636969.1
NC_035780.1	Gnomon	exon	14557	14594	.	+	.	ID=exon-XR_002636969.1-3;Parent=rna-XR_002636969.1;Dbxref=GeneID:111116054,Genbank:XR_002636969.1;gbkey=ncRNA;gene=LOC111116054;product=uncharacterized LOC111116054;transcript_id=XR_002636969.1
NC_035780.1	Gnomon	exon	28961	29073	.	+	.	ID=exon-XM_022471938.1-1;Parent=rna-XM_022471938.1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;gbkey=mRNA;gene=LOC111126949;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	Gnomon	exon	30524	31557	.	+	.	ID=exon-XM_022471938.1-2;

### 2d. lncRNA

In [44]:
!grep -e "Gnomon	lnc_RNA" -e "RefSeq	lnc_RNA" -e "tRNAscan-SE	lnc_RNA" \
GCF_002022765.2_C_virginica-3.0_genomic.gff \
| {bedtoolsDirectory}/sortBed \
-faidx C_virginica-3.0-chr.txt \
> C_virginica-3.0-lncRNA.gff

In [45]:
!head C_virginica-3.0-lncRNA.gff
!wc -l C_virginica-3.0-lncRNA.gff

NC_035780.1	Gnomon	lnc_RNA	13578	14594	.	+	.	ID=rna-XR_002636969.1;Parent=gene-LOC111116054;Dbxref=GeneID:111116054,Genbank:XR_002636969.1;Name=XR_002636969.1;gbkey=ncRNA;gene=LOC111116054;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 1 sample with support for all annotated introns;product=uncharacterized LOC111116054;transcript_id=XR_002636969.1
NC_035780.1	Gnomon	lnc_RNA	169468	170178	.	-	.	ID=rna-XR_002635081.1;Parent=gene-LOC111105702;Dbxref=GeneID:111105702,Genbank:XR_002635081.1;Name=XR_002635081.1;gbkey=ncRNA;gene=LOC111105702;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 3 samples with support for all annotated introns;product=uncharacterized LOC111105702;transcript_id=XR_002635081.1
NC_035780.1	Gnomon	lnc_RNA	900326	903430	.	+	.	ID=rna-XR_002636046.1;Parent=gene-LOC111111519;Dbxref=GeneID

### 2e. mRNA

In [46]:
!grep -e "Gnomon	mRNA" -e "RefSeq	mRNA" -e "tRNAscan-SE	mRNA" \
GCF_002022765.2_C_virginica-3.0_genomic.gff \
| {bedtoolsDirectory}/sortBed \
-faidx C_virginica-3.0-chr.txt \
> C_virginica-3.0-mRNA.gff

In [47]:
!head C_virginica-3.0-mRNA.gff
!wc -l C_virginica-3.0-mRNA.gff

NC_035780.1	Gnomon	mRNA	28961	33324	.	+	.	ID=rna-XM_022471938.1;Parent=gene-LOC111126949;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	Gnomon	mRNA	43111	66897	.	-	.	ID=rna-XM_022447324.1;Parent=gene-LOC111110729;Dbxref=GeneID:111110729,Genbank:XM_022447324.1;Name=XM_022447324.1;gbkey=mRNA;gene=LOC111110729;model_evidence=Supporting evidence includes similarity to: 1 Protein%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments;product=FMRFamide receptor-like%2C transcript variant X1;transcript_id=XM_022447324.1
NC_035780.1	Gnomon	mRNA	43111	46506	.	-	.	ID=rna-XM_022447333.1;Parent=gene-LOC111110729;Dbxref=GeneID:111110729,Genbank:XM_02244733

### 2f. Non-coding sequences

In [48]:
#Find the complement to the exon track (non-coding sequences)
#Create a BEDfile of IGV
!{bedtoolsDirectory}/complementBed \
-i C_virginica-3.0-exon.gff \
-g C_virginica-3.0-sequence-lengths.txt \
> C_virginica-3.0-nonCDS.bed

In [49]:
!head C_virginica-3.0-nonCDS.bed
!wc -l C_virginica-3.0-nonCDS.bed

NC_035780.1	0	13577
NC_035780.1	13603	14236
NC_035780.1	14290	14556
NC_035780.1	14594	28960
NC_035780.1	29073	30523
NC_035780.1	31557	31735
NC_035780.1	31887	31976
NC_035780.1	32565	32958
NC_035780.1	33324	43110
NC_035780.1	44358	45912
  337305 C_virginica-3.0-nonCDS.bed


### 2g. Intron

In [53]:
#Find the intersection between the non-coding sequences and genes (introns)
!{bedtoolsDirectory}/intersectBed \
-a C_virginica-3.0-nonCDS.bed \
-b C_virginica-3.0-gene.gff -sorted \
> C_virginica-3.0-intron.bed

In [54]:
!head C_virginica-3.0-intron.bed
!wc -l C_virginica-3.0-intron.bed

NC_035780.1	13603	14236
NC_035780.1	14290	14556
NC_035780.1	29073	30523
NC_035780.1	31557	31735
NC_035780.1	31887	31976
NC_035780.1	32565	32958
NC_035780.1	44358	45912
NC_035780.1	46506	64122
NC_035780.1	64334	66868
NC_035780.1	85777	88422
  311341 C_virginica-3.0-intron.bed


### 2h. Untranslated regions of exons

In [184]:
#Obtain UTRs by subtracting CDS from exons
!{bedtoolsDirectory}subtractBed \
-a cgigas_uk_roslin_v1_exon.gff \
-b cgigas_uk_roslin_v1_CDS.gff \
-sorted \
-g cgigas_uk_roslin_v1-sequence-lengths.txt \
> cgigas_uk_roslin_v1_exonUTR.gff

In [185]:
!head cgigas_uk_roslin_v1_exonUTR.gff
!wc -l cgigas_uk_roslin_v1_exonUTR.gff

NC_047559.1	Gnomon	exon	9839	9922	.	+	.	ID=exon-XR_004604272.1-1;Parent=rna-XR_004604272.1;Dbxref=GeneID:117693020,Genbank:XR_004604272.1;gbkey=ncRNA;gene=LOC117693020;product=uncharacterized LOC117693020;transcript_id=XR_004604272.1
NC_047559.1	Gnomon	exon	10884	11386	.	+	.	ID=exon-XR_004604272.1-2;Parent=rna-XR_004604272.1;Dbxref=GeneID:117693020,Genbank:XR_004604272.1;gbkey=ncRNA;gene=LOC117693020;product=uncharacterized LOC117693020;transcript_id=XR_004604272.1
NC_047559.1	Gnomon	exon	14114	14545	.	+	.	ID=exon-XM_034463183.1-1;Parent=rna-XM_034463183.1;Dbxref=GeneID:109621113,Genbank:XM_034463183.1;gbkey=mRNA;gene=LOC109621113;product=uncharacterized LOC109621113;transcript_id=XM_034463183.1
NC_047559.1	Gnomon	exon	14812	14842	.	+	.	ID=exon-XM_034463183.1-2;Parent=rna-XM_034463183.1;Dbxref=GeneID:109621113,Genbank:XM_034463183.1;gbkey=mRNA;gene=LOC109621113;product=uncharacterized LOC109621113;transcript_id=XM_034463183.1
NC_047559.1	Gnomon	exon	18981	19160	.	-	.	ID=exon-XM_0344631

### 2i. Flanking regions (1 kb)

#### All flanks

In [237]:
#Create 1 kb flanking regions
#Subtract existing genes from artificial flanks
!{bedtoolsDirectory}flankBed \
-i cgigas_uk_roslin_v1_gene.gff \
-g cgigas_uk_roslin_v1-sequence-lengths.txt \
-b 1000 \
| {bedtoolsDirectory}subtractBed \
-a - \
-b cgigas_uk_roslin_v1_gene.gff \
> cgigas_uk_roslin_v1_flanks.gff

In [238]:
!head cgigas_uk_roslin_v1_flanks.gff
!wc -l cgigas_uk_roslin_v1_flanks.gff

NC_047559.1	Gnomon	gene	8839	9838	.	+	.	ID=gene-LOC117693020;Dbxref=GeneID:117693020;Name=LOC117693020;gbkey=Gene;gene=LOC117693020;gene_biotype=lncRNA
NC_047559.1	Gnomon	gene	11387	12386	.	+	.	ID=gene-LOC117693020;Dbxref=GeneID:117693020;Name=LOC117693020;gbkey=Gene;gene=LOC117693020;gene_biotype=lncRNA
NC_047559.1	Gnomon	gene	13114	14113	.	+	.	ID=gene-LOC109621113;Dbxref=GeneID:109621113;Name=LOC109621113;gbkey=Gene;gene=LOC109621113;gene_biotype=protein_coding
NC_047559.1	Gnomon	gene	15805	16804	.	+	.	ID=gene-LOC109621113;Dbxref=GeneID:109621113;Name=LOC109621113;gbkey=Gene;gene=LOC109621113;gene_biotype=protein_coding
NC_047559.1	Gnomon	gene	15867	16866	.	-	.	ID=gene-LOC117687066;Dbxref=GeneID:117687066;Name=LOC117687066;gbkey=Gene;gene=LOC117687066;gene_biotype=protein_coding
NC_047559.1	Gnomon	gene	19161	20160	.	-	.	ID=gene-LOC117687066;Dbxref=GeneID:117687066;Name=LOC117687066;gbkey=Gene;gene=LOC117687066;gene_biotype=protein_coding
NC_047559.1	Gnomon	gene	59036	60035	.	-	.	ID=g

#### Upstream flanks

In [16]:
#Create 1 kb upstream flanking regions (-l) based on strand (-s)
#Subtract existing genes from artificial flanks
!{bedtoolsDirectory}flankBed \
-i cgigas_uk_roslin_v1_gene.gff \
-g cgigas_uk_roslin_v1-sequence-lengths.txt \
-l 1000 \
-r 0 \
-s \
| {bedtoolsDirectory}subtractBed \
-a - \
-b cgigas_uk_roslin_v1_gene.gff \
> cgigas_uk_roslin_v1_upstream.gff

In [17]:
!head cgigas_uk_roslin_v1_upstream.gff
!wc -l cgigas_uk_roslin_v1_upstream.gff

NC_047559.1	Gnomon	gene	8839	9838	.	+	.	ID=gene-LOC117693020;Dbxref=GeneID:117693020;Name=LOC117693020;gbkey=Gene;gene=LOC117693020;gene_biotype=lncRNA
NC_047559.1	Gnomon	gene	13114	14113	.	+	.	ID=gene-LOC109621113;Dbxref=GeneID:109621113;Name=LOC109621113;gbkey=Gene;gene=LOC109621113;gene_biotype=protein_coding
NC_047559.1	Gnomon	gene	19161	20160	.	-	.	ID=gene-LOC117687066;Dbxref=GeneID:117687066;Name=LOC117687066;gbkey=Gene;gene=LOC117687066;gene_biotype=protein_coding
NC_047559.1	Gnomon	gene	74624	75623	.	-	.	ID=gene-LOC117689737;Dbxref=GeneID:117689737;Name=LOC117689737;gbkey=Gene;gene=LOC117689737;gene_biotype=protein_coding
NC_047559.1	Gnomon	gene	79321	80320	.	+	.	ID=gene-LOC117687025;Dbxref=GeneID:117687025;Name=LOC117687025;gbkey=Gene;gene=LOC117687025;gene_biotype=protein_coding
NC_047559.1	Gnomon	gene	96820	97819	.	+	.	ID=gene-LOC105333279;Dbxref=GeneID:105333279;Name=LOC105333279;gbkey=Gene;gene=LOC105333279;gene_biotype=protein_coding
NC_047559.1	Gnomon	gene	141528	142527	

#### Downstream flanks

In [18]:
#Create 1 kb upstream flanking regions (-l) based on strand (-s)
#Subtract existing genes from artificial flanks
!{bedtoolsDirectory}flankBed \
-i cgigas_uk_roslin_v1_gene.gff \
-g cgigas_uk_roslin_v1-sequence-lengths.txt \
-l 0 \
-r 1000 \
-s \
| {bedtoolsDirectory}subtractBed \
-a - \
-b cgigas_uk_roslin_v1_gene.gff \
> cgigas_uk_roslin_v1_downstream.gff

In [19]:
!head cgigas_uk_roslin_v1_downstream.gff
!wc -l cgigas_uk_roslin_v1_downstream.gff

NC_047559.1	Gnomon	gene	11387	12386	.	+	.	ID=gene-LOC117693020;Dbxref=GeneID:117693020;Name=LOC117693020;gbkey=Gene;gene=LOC117693020;gene_biotype=lncRNA
NC_047559.1	Gnomon	gene	15805	16804	.	+	.	ID=gene-LOC109621113;Dbxref=GeneID:109621113;Name=LOC109621113;gbkey=Gene;gene=LOC109621113;gene_biotype=protein_coding
NC_047559.1	Gnomon	gene	15867	16866	.	-	.	ID=gene-LOC117687066;Dbxref=GeneID:117687066;Name=LOC117687066;gbkey=Gene;gene=LOC117687066;gene_biotype=protein_coding
NC_047559.1	Gnomon	gene	59036	60035	.	-	.	ID=gene-LOC117689737;Dbxref=GeneID:117689737;Name=LOC117689737;gbkey=Gene;gene=LOC117689737;gene_biotype=protein_coding
NC_047559.1	Gnomon	gene	86125	87124	.	+	.	ID=gene-LOC117687025;Dbxref=GeneID:117687025;Name=LOC117687025;gbkey=Gene;gene=LOC117687025;gene_biotype=protein_coding
NC_047559.1	Gnomon	gene	126388	127387	.	+	.	ID=gene-LOC105333279;Dbxref=GeneID:105333279;Name=LOC105333279;gbkey=Gene;gene=LOC105333279;gene_biotype=protein_coding
NC_047559.1	Gnomon	gene	138293	139

### 2j. Intergenic regions

In [4]:
#Find the complement of genes, then subtract flanks to obtain intergenic regions
!{bedtoolsDirectory}complementBed \
-i cgigas_uk_roslin_v1_gene.gff -sorted \
-g cgigas_uk_roslin_v1-sequence-lengths.txt \
| {bedtoolsDirectory}subtractBed \
-a - \
-b cgigas_uk_roslin_v1_flanks.gff \
> cgigas_uk_roslin_v1_intergenic.bed

In [5]:
!head cgigas_uk_roslin_v1_intergenic.bed
!wc -l cgigas_uk_roslin_v1_intergenic.bed

NC_047559.1	0	8838
NC_047559.1	12386	13113
NC_047559.1	20160	59035
NC_047559.1	75623	79320
NC_047559.1	87124	96819
NC_047559.1	127387	138292
NC_047559.1	147045	150757
NC_047559.1	186673	202631
NC_047559.1	210757	225702
NC_047559.1	230170	241188
   21598 cgigas_uk_roslin_v1_intergenic.bed


### 2k. Transposable elements

In [240]:
!curl https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/902/806/645/GCF_902806645.1_cgigas_uk_roslin_v1/GCF_902806645.1_cgigas_uk_roslin_v1_rm.out.gz \
> cgigas_uk_roslin_v1_rm.te.gz 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24.5M  100 24.5M    0     0   9.8M      0  0:00:02  0:00:02 --:--:-- 10.1M


In [241]:
!gunzip -k cgigas_uk_roslin_v1_rm.te.gz

In [242]:
!head cgigas_uk_roslin_v1_rm.te

   SW     perc    perc    perc  query           position in query                   matching       repeat           position in repeat
score     div.    del.    ins.  sequence        begin      end             (left)   repeat         class/family   begin  end    (left)     ID

   18    0.000   0.000   0.000  NC_047559.1           4109       4128  (55781200) + (C)n           Simple_repeat       1     20    (0)      1
   20    4.200   0.000   0.000  NC_047559.1           5159       5183  (55780145) + (AG)n          Simple_repeat       1     25    (0)      2
   22    3.800   0.000   0.000  NC_047559.1           5546       5572  (55779756) + (AG)n          Simple_repeat       1     27    (0)      3
   15   14.700   0.000   0.000  NC_047559.1           5928       5957  (55779371) + (GAGA)n        Simple_repeat       1     30    (0)      4
  218    0.500   0.500   0.000  NC_047559.1           6774       6968  (55778360) + (CCCTAA)n      Simple_repeat       1    196    (0)      5
  105    0.0

In [11]:
#Convert RepeatMasker output to a BEDfile
#Skip the first 4 lines 
#Print columns 5-7 as a tab-delimited output
!tail -n +4 cgigas_uk_roslin_v1_rm.te \
| awk 'BEGIN{OFS= "\t"} {print $5, $6, $7}' \
> cgigas_uk_roslin_v1_rm.te.bed

In [13]:
!head cgigas_uk_roslin_v1_rm.te.bed
!wc -l cgigas_uk_roslin_v1_rm.te.bed

NC_047559.1	4109	4128
NC_047559.1	5159	5183
NC_047559.1	5546	5572
NC_047559.1	5928	5957
NC_047559.1	6774	6968
NC_047559.1	9671	9766
NC_047559.1	15714	15759
NC_047559.1	19137	19509
NC_047559.1	20983	21150
NC_047559.1	22619	22836
  781312 cgigas_uk_roslin_v1_rm.te.bed


## 3. CG motifs

### 3a. Count CGs

In [1]:
#Somatic and mitochondrial genome
!head /Volumes/web/spartina/project-oyster-oa/data/Cg-roslin/cgigas_uk_roslin_v1_genomic-mito.fa

>NC_047559.1 Crassostrea gigas linkage group 1, cgigas_uk_roslin_v1, whole genome shotgun sequence
AAATAATCGGTGTTAATTTAAGGACAAATATTTCTATAAAGAAATTACATCCCTGGAAAGAAAccaatattcttcatttt
aatgttaaaataggGATATAGTGTAAATGCTCCCGAAAGCCCCCTAAAACTTTTGGGATCAATAATAACACCCGTATTTC
TCCGTTgatacattaatacaaagagAACAAAGAAATCATTAGCATTAGTTTcagataaaattctttataaaaaaaattac
acccctgagaaaactcaaaatattctgtttagcgttaaaaaaaatcagtagacTTAATTAGTTgggaaactccctaaatc
ttttgataTGAATACTTACACCCTTATTTTATCTTGATTACATGAATacgaagaaataaacaaaatcatgttaaatgaaa
aatcactttattaaaaatgacatcctcaggaaggaacccatattcttcattttaacgttcaatAATAGGTATAGTTTAAA
CGCTAAGTAGACCCCCAAAAACTTTTGGGATTAAGACTTACACCATCAATTCTCCTTTAGtgcattaaaacaaagaaaat
aatcagctttagttttaggaaaaatccttttattgagaaataacacCCCTGAGAAtactcaaaattttcattttattgtt
aaaaaaatccgCTAGGGTAGTAAATCAGTTAGGAAACTCCTTAATtatcttttggtatgaataccTACACCCTTATATAT


In [2]:
#Obtain a rough count of CGs in the genome
!fgrep -o -i CG /Volumes/web/spartina/project-oyster-oa/data/Cg-roslin/cgigas_uk_roslin_v1_genomic-mito.fa \
| wc -l

 13080574


Generated CG motif track with `fuzznuc` on Galaxy

![Screen Shot 2021-05-11 at 12 19 30 PM](https://user-images.githubusercontent.com/22335838/117872110-2522cf00-b253-11eb-8766-a5cb7c15515d.png)

In [7]:
#Check Galaxy output
!head cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff
!wc -l cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff

##gff-version 2.0
##date 2021-05-11
##Type DNA NC_047559.1
NC_047559.1	fuzznuc	misc_feature	8	9	2.000	+	.	Sequence "NC_047559.1.1" ; note "*pat pattern1"
NC_047559.1	fuzznuc	misc_feature	114	115	2.000	+	.	Sequence "NC_047559.1.2" ; note "*pat pattern1"
NC_047559.1	fuzznuc	misc_feature	153	154	2.000	+	.	Sequence "NC_047559.1.3" ; note "*pat pattern1"
NC_047559.1	fuzznuc	misc_feature	163	164	2.000	+	.	Sequence "NC_047559.1.4" ; note "*pat pattern1"
NC_047559.1	fuzznuc	misc_feature	273	274	2.000	+	.	Sequence "NC_047559.1.5" ; note "*pat pattern1"
NC_047559.1	fuzznuc	misc_feature	369	370	2.000	+	.	Sequence "NC_047559.1.6" ; note "*pat pattern1"
NC_047559.1	fuzznuc	misc_feature	456	457	2.000	+	.	Sequence "NC_047559.1.7" ; note "*pat pattern1"
 13246547 cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff


### 3b. Count CG overlaps with all genome feature tracks

In [11]:
#Genes
!{bedtoolsDirectory}intersectBed \
-u \
-a cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff \
-b cgigas_uk_roslin_v1_gene.gff \
| wc -l

 7425225


In [12]:
#CDS
!{bedtoolsDirectory}intersectBed \
-u \
-a cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff \
-b cgigas_uk_roslin_v1_CDS.gff \
| wc -l

 1516641


In [13]:
#Exon
!{bedtoolsDirectory}intersectBed \
-u \
-a cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff \
-b cgigas_uk_roslin_v1_exon.gff \
| wc -l

 2216766


In [14]:
#lncRNA
!{bedtoolsDirectory}intersectBed \
-u \
-a cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff \
-b cgigas_uk_roslin_v1_lncRNA.gff \
| wc -l

  454393


In [15]:
#mRNA
!{bedtoolsDirectory}intersectBed \
-u \
-a cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff \
-b cgigas_uk_roslin_v1_mRNA.gff \
| wc -l

 7040925


In [16]:
#nonCDS
!{bedtoolsDirectory}intersectBed \
-u \
-a cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff \
-b cgigas_uk_roslin_v1_nonCDS.bed \
| wc -l

 11040607


In [17]:
#Introns
!{bedtoolsDirectory}intersectBed \
-u \
-a cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff \
-b cgigas_uk_roslin_v1_intron.bed \
| wc -l

 5221932


In [18]:
#Exon UTR
!{bedtoolsDirectory}intersectBed \
-u \
-a cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff \
-b cgigas_uk_roslin_v1_exonUTR.gff \
| wc -l

  700167


In [19]:
#Flanking regions
!{bedtoolsDirectory}intersectBed \
-u \
-a cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff \
-b cgigas_uk_roslin_v1_flanks.gff \
| wc -l

 1091847


In [20]:
#Upstream flanks
!{bedtoolsDirectory}intersectBed \
-u \
-a cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff \
-b cgigas_uk_roslin_v1_upstream.gff \
| wc -l

  592834


In [21]:
#Downstream flanks
!{bedtoolsDirectory}intersectBed \
-u \
-a cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff \
-b cgigas_uk_roslin_v1_downstream.gff \
| wc -l

  549787


In [22]:
#Intergenic regions
!{bedtoolsDirectory}intersectBed \
-u \
-a cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff \
-b cgigas_uk_roslin_v1_intergenic.bed \
| wc -l

 4730728


In [23]:
#Transposable elements
!{bedtoolsDirectory}intersectBed \
-u \
-a cgigas_uk_roslin_v1_fuzznuc_CGmotif.gff \
-b cgigas_uk_roslin_v1_rm.te.bed \
| wc -l

 5424483
